## Q1. Chi Square Test

In [1]:
#TODO


In [2]:
#TODO
conclusions = 'your conclusions'
print('Conclusions:\t', conclusions)

Conclusions:	 your conclusions


## Q2. Correlation coefficients

In [3]:
#TODO pearson coefficient


In [4]:
#TODO spearman coefficient


In [5]:
#TODO
conclusions = 'your conclusions'
print('Conclusions:\t', conclusions)

Conclusions:	 your conclusions


## Q3. Association rule

In [6]:
#TODO


In [7]:
#TODO
conclusions = 'your conclusions'
print('Conclusions:\t', conclusions)

Conclusions:	 your conclusions


## Q4. Power law distribution

In [8]:
#TODO ccdf


In [9]:
#TODO powerlaw


In [10]:
#TODO
conclusions = 'your conclusions'
print('Conclusions:\t', conclusions)

Conclusions:	 your conclusions
